In [1]:
# necessary libraries for prediciton
import utils
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVR

# Prediction

* Predictions are done in this notebook.
* It is also useful to compare how serveral algorithms perform against one another.

In [12]:
dataset =  utils.read_csv_to_df('dataset/preprocessed_data.csv')
display(dataset)

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,-1
1,5316,1801,930711,165960,36,4610,1
2,6863,9188,930728,127080,60,2118,1
3,5325,1843,930803,105804,36,2939,1
4,7240,11013,930906,274740,60,4579,1
...,...,...,...,...,...,...,...
323,6818,9030,961212,155616,48,3242,1
324,5625,3189,961215,222180,60,3703,-1
325,6805,8972,961221,45024,48,938,1
326,7233,10963,961225,115812,36,3217,1


In [13]:
# Useful MACROS
STATUS_COL = 6
K_FOLD_NUM_SPLITS = 5
SEED = 42

In [26]:
# Setting X and Y
X = dataset.iloc[:, 0:STATUS_COL-1]
Y = dataset.iloc[:, [STATUS_COL]]

In [38]:
# Cross validation settings
scores = []
cv = KFold(n_splits=K_FOLD_NUM_SPLITS, random_state=SEED, shuffle=False)

# CHANGE THIS LINE TO CHANGE THE USED CLASSIFICATION METHOD


# Applying Cross validation
for train_index, test_index in cv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    # best_svr.fit(X_train, Y_train)
    # scores.append(best_svr.score(X_test, Y_test))